# Prerequisites

## For Server 
- git clone https://github.com/most-medicalrecord -b develop
- cd most-medicalrecord
- make develop
- make sync
- make run

##For testing API with this notebook## 
- **Requests** library <http://docs.python-requests.org/en/latest/>

## For testing pyEHR API endpoint
- Download ova VirtualMachine from <http://156.148.18.70/files/pyEHR.ova>
- Run VirtualMachine (virtualbox or vmware importing vmdk virtual disk in new virtual machine)
- Configure VirtualMachine networking with NAT or BRIDGE
- Wait for services start (up to 5 minutes)
- Test dbservice status calling <http://virtualmachine:8080/check/status/dbservice>
- Configure PYEHR_DB_SERVICE_IP on django settings.py
- stop SERVER and restart with **make run**

In [1]:
#Get Oauth2 Access Token specifying existing taskgroup_uuid

import requests
import json

server_port = 9000

arguments = {
             'client_id': '8c96bf8cea26fa555fa8',
             'client_secret': '4fd1f508b7b03fba6509da4c193157d7a2b20838',
             'grant_type': 'password',
             'username': 'admin',
             'password': 'admin',
             'taskgroup': '5dw2x3jfkftxue5a5izw6yiplbbn4dlo'
             }
r = requests.post(
                  'http://127.0.0.1:{server_port}/oauth2/access_token/'.format(server_port=server_port), 
                  data=arguments
                  )
print r.text

access_token = json.loads(r.text)['access_token']

{"access_token": "2362afb4f2ec78bd0bdc5a4bfdda572962942b2e", "token_type": "Bearer", "expires_in": 2591999, "scope": "read"}


In [2]:
#Get EHR DBService status

payload = {'access_token': access_token}
r = requests.get(
    'http://127.0.0.1:{server_port}/medicalrecords/ehr/status/'.format(server_port=server_port), 
    params=payload)
print r.text

DBService daemon running


In [3]:
#Create patient record
payload = {'access_token': access_token}

r = requests.post(
    'http://127.0.0.1:{server_port}/medicalrecords/patients/'.format(server_port=server_port), 
    params=payload
)
print r.text
data = json.loads(r.text)
patient_id = data['patient']['uuid']
print 'patient id: %s' % patient_id

{"patient": {"demographic_uuid": "ipn7fbtidx3drht27zhng4eibji7icj6", "ehr_uuid": "6w363wxy6lfzomnc2lczbrandc5yws74", "uuid": "iq6aabj7xvqml6h4hiqntkc2lyhi7huj"}, "success": true}
patient id: iq6aabj7xvqml6h4hiqntkc2lyhi7huj


In [4]:
#Create ehr patient record 
payload = {'access_token': access_token}

r = requests.post(
    'http://127.0.0.1:{server_port}/medicalrecords/ehr/{patient_id}/'.
    format(server_port=server_port, patient_id=patient_id), 
    params=payload
)
print r.text

{"RECORD": {"record_id": "6w363wxy6lfzomnc2lczbrandc5yws74", "active": true, "ehr_records": [], "creation_time": 1490371566.508712, "last_update": 1490371566.508712}, "SUCCESS": true}


In [6]:
#Create ehr medical record
params = {'access_token': access_token}

ehr_data = {
  "archetype_class": "openEHR.TEST-EVALUATION.v1",
  "archetype_details": {
    "at0001": "val1",
    "at0002": "val2"
  }
}

r = requests.post(
    'http://127.0.0.1:{server_port}/medicalrecords/ehr/{patient_id}/records/'.
        format(server_port=server_port, patient_id=patient_id), 
    params=params, json=ehr_data
)
print r.text

{"RECORD": {"ehr_data": {"archetype_details": {"at0001": "val1", "at0002": "val2"}, "archetype_class": "openEHR.TEST-EVALUATION.v1"}, "creation_time": 1490371656.850536, "patient_id": "6w363wxy6lfzomnc2lczbrandc5yws74", "record_id": "a73594e16ac94b6f9dd788794a849495", "version": 1, "active": true, "last_update": 1490371656.850536}, "SUCCESS": true}


In [7]:
#Check new EHR entry for current medicalrecord patient
payload = {'access_token': access_token}
r = requests.get(
                    'http://127.0.0.1:{server_port}/medicalrecords/ehr/{patient_id}/'.
                    format(server_port=server_port, patient_id=patient_id), 
                    params=payload
                  )
data = json.loads(r.text)
print 'number of records: %s ' % len(data['RECORD'])
record_id = data['RECORD']['ehr_records'][-1]['record_id']
print 'id of last record: %s' % record_id
print data['RECORD']['ehr_records'][-1]

number of records: 5 
id of last record: a73594e16ac94b6f9dd788794a849495
{u'ehr_data': {u'archetype_details': {}, u'archetype_class': u'openEHR.TEST-EVALUATION.v1'}, u'creation_time': 1490371656.850536, u'patient_id': u'6w363wxy6lfzomnc2lczbrandc5yws74', u'record_id': u'a73594e16ac94b6f9dd788794a849495', u'version': 1, u'active': True, u'last_update': 1490371656.850536}


In [8]:
#Get new EHR record
payload = {'access_token': access_token}
r = requests.get(
                    'http://127.0.0.1:{server_port}/medicalrecords/ehr/{patient_id}/records/{record_id}/'.
                    format(server_port=server_port, patient_id=patient_id, record_id=record_id), 
                    params=payload
                  )
data = json.loads(r.text)
print data

{u'RECORD': {u'ehr_data': {u'archetype_details': {u'at0001': u'val1', u'at0002': u'val2'}, u'archetype_class': u'openEHR.TEST-EVALUATION.v1'}, u'creation_time': 1490371656.850536, u'patient_id': u'6w363wxy6lfzomnc2lczbrandc5yws74', u'active': True, u'version': 1, u'record_id': u'a73594e16ac94b6f9dd788794a849495', u'last_update': 1490371656.850536}, u'SUCCESS': True}


In [10]:
#Create and delete new ehr medical record
params = {'access_token': access_token}

ehr_data = {
  "archetype_class": "openEHR.TEST-EVALUATION.v1",
  "archetype_details": {
    "at0001": "val1",
    "at0002": "val2"
  }
}

r = requests.post(
    'http://127.0.0.1:{server_port}/medicalrecords/ehr/{patient_id}/records/'.
    format(server_port=server_port, patient_id=patient_id), 
    params=params, json=ehr_data
)
data = json.loads(r.text)
record_id = data['RECORD']['record_id']

payload = {'access_token': access_token}
r = requests.delete(
    'http://127.0.0.1:{server_port}/medicalrecords/ehr/{patient_id}/records/{record_id}/'.
        format(server_port=server_port, patient_id=patient_id, record_id=record_id), 
    params=payload, data={})

print r.text

{"MESSAGE": "EHR record with ID 9d0bc867a3484bea8655356fc34aac13 successfully hidden", "SUCCESS": true}


In [12]:
#Create and delete patient, and ehr patient

#Create medical record patient
payload = {'access_token': access_token}

r = requests.post(
    'http://127.0.0.1:{server_port}/medicalrecords/patients/'.format(server_port=server_port), 
    params=payload
)
print r.text
data = json.loads(r.text)
patient_id = data['patient']['uuid']

#Create ehr patient record 
payload = {'access_token': access_token}

r = requests.post(
    'http://127.0.0.1:{server_port}/medicalrecords/ehr/{patient_id}/'.
    format(server_port=server_port, patient_id=patient_id), 
params=payload)
print r.text

#Delete ehr patient record

r = requests.delete(
    'http://127.0.0.1:{server_port}/medicalrecords/ehr/{patient_id}/'.
    format(server_port=server_port, patient_id=patient_id), 
params=payload, data={})

print r.text

{"patient": {"demographic_uuid": "v3oxhina4qg36edwuah2pkdrebzpil2g", "ehr_uuid": "mhzchxpmmamhfsv67mdsztx3xhwy2eh7", "uuid": "vf2sd45trfsflns42srjjdsf2kgwleox"}, "success": true}
{"RECORD": {"record_id": "mhzchxpmmamhfsv67mdsztx3xhwy2eh7", "active": true, "ehr_records": [], "creation_time": 1490371966.29505, "last_update": 1490371966.29505}, "SUCCESS": true}
{"RECORD": {"record_id": "mhzchxpmmamhfsv67mdsztx3xhwy2eh7", "active": false, "ehr_records": [], "creation_time": 1490371966.29505, "last_update": 1490371966.428442}, "SUCCESS": true}
